<a href="https://colab.research.google.com/github/Mohadesehzarei/skqulacs_QSVM/blob/main/skqulacs_QSVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Potential of quantum machine learning for solving the real-world problem of cancers classification

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from skfeature.function.similarity_based import fisher_score
from imblearn.over_sampling import SMOTE
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
import itertools
import matplotlib.pyplot as plt
%matplotlib inline

#QSVM
from skqulacs.circuit.pre_defined import create_yzcx_ansatz
from qulacs import QuantumCircuit
from qulacsvis import circuit_drawer
from skqulacs.qsvm import QSVC

# Expression_DEGs

In [ ]:
#import Data
X = #expression Data
y = #class

In [ ]:
# split data to train and test
x_train, x_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=0
)
x_train_=x_train
y_train=np.array(y_train)
y_test=np.array(y_test)
x_train = x_train.to_numpy()

# Fisher score algorithm

In [ ]:
def fisher_score(X, y):
    num_features = X.shape[1]
    num_classes = len(np.unique(y))

    class_means = np.zeros((num_classes, num_features))
    class_variances = np.zeros((num_classes, num_features))

    for label in range(num_classes):
        class_data = X[y == label]
        class_means[label] = np.mean(class_data, axis=0)
        class_variances[label] = np.var(class_data, axis=0)

    grand_mean = np.mean(X, axis=0)

    numerator = np.sum((class_means - grand_mean) ** 2, axis=0)
    denominator = np.sum(class_variances, axis=0)

    fisher_scores = numerator / denominator

    return fisher_scores

In [ ]:
sel_cols = SelectKBest(fisher_score, k=5)
sel_cols.fit(x_train_, y_train_new)
sel = x_train_.columns[sel_cols.get_support()]
indices = x_train_.columns.get_indexer(sel)
x_train_sorted_ = x_train_.iloc[:,indices]
x_test_sorted_ = x_test.iloc[:,indices]
x_test_sorted = x_test_sorted_.to_numpy()

# Quantum Machine Learning (skqulacs-QSVM)

In [ ]:
from imblearn.over_sampling import SMOTE
smt=SMOTE(k_neighbors=2,random_state=42)
x_train_SMOTE_,y_train_SMOTE_=smt.fit_resample(x_train_sorted_, y_train)
#x_test_SMOTE,y_test_SMOTE=smt.fit_resample(x_test, y_test)
c_depth = 1
time_step = 1

In [ ]:
i= #optimum number of features with the highest accuracy

X_train_SMOTE=x_train_SMOTE_.iloc[:,0:i]
X_train_SMOTE = X_train_SMOTE.to_numpy()
y_train_SMOTE=np.array(y_train_SMOTE_)

x_test_sorted=x_test_sorted_.iloc[:,0:i]
x_test_sorted = x_test_sorted.to_numpy()

In [ ]:
from sklearn.model_selection import KFold, cross_val_score
start = time.time()
n_qubit=5

x_train_SMOTE = x_train_SMOTE_.to_numpy()
y_train_SMOTE=np.array(y_train_SMOTE_)
# Requires at least the number of dimensions of x_train. If it is too small, the result will be bad.
circuit_yzcx = create_yzcx_ansatz(n_qubit, c_depth)
qsvm = QSVC(circuit_yzcx)
model=qsvm.fit(x_train_SMOTE, y_train_SMOTE)
y_pred = qsvm.predict(x_test_sorted)
accuracy=accuracy_score(y_test, y_pred)

df=classification_report(y_test,y_pred)
print(df)
print(accuracy)

elapsed = time.time() - start

In [ ]:
# plot circuit
circuit_drawer(circuit_yzcx._circuit, "mpl")
plt.show()
